# Lab 6: Random processes and variations on a random walk

At this point, you should recognize the need for this code. We are bringing in the modules necessary for this lab.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats as st
print ("Modules Imported!")

## Random Processes:

Before we go over random walks, it's first a good idea to talk about a random process.  Simply put, a random process is a collection of indexed random variables on the same probability space. We say $X = (X_t: t\in \cal T)$ is a *random process* where each $X_t$ is a random variable. The index $t$ often corresponds to time.  We already saw one type of random process, namely, a Bernoulli random process with parameter $p$, such that $T$ is the positive integers and $X_t$ is a Bernoulli random variable with parameter $p$ for each $t\geq 1.$  A random walk is a path through time based on discrete random steps. After any given number of steps, the next step is determined by a random variable. Let's visualize this with a scenario. A man goes on a walk with a bag full of marbles. Inside the bag are 4 blue marbles, 3 red marbles, and 2 yellow marble. At every point in time the man pulls a marble out of the bag to determine his next step. If he pulls a blue marble he takes one step back, a red, one step forward, and a yellow two steps forward. After taking the step he places the marble back in the bag.  Below is a graph of this behavior for 40 time units. Each step he takes is a random variable based on his pick of marbles.

In [ ]:
p = [i*1./(4+3+2) for i in [4,3,2]]
c = [-1,1,2]
T = 40   # Here T denotes the number of time steps
Xmarb = st.rv_discrete(name='Xmarb',values=(c,p))

x = [0]
for t in range(T):
    x.append(x[t]+Xmarb.rvs())
    
plt.plot(x, 'ro', label = 'Walking Step')
t = np.linspace(0,T,10000)
s = Xmarb.mean()  # s is the mean of one step
plt.plot(t,np.multiply(s,t), label='Average Step')
plt.title('Random Walk')
plt.xlabel('Time Step')
plt.ylabel('Position')
plt.legend(loc="best")

In the above graph we plot the actual path the man takes versus what we would expect his path to be. At each step he independently pulls out a marble. We can calculate his expected step size by simply finding the expected value of the random variable for one step. If you increase the number of time steps $T$, you'll notice that his random walk more closely resembles our expectations. This follows directly from the *law of large numbers.*

In this next example, we do the same thing, but now the probability distributions of his steps vary with time. That is, for for each step we create a new distribution. For every unit in time, the man can either take two steps forward with probability $\cos^2 (2\pi n/T)$ or take one step backward with probability  $\sin^2(2\pi n/T)$. So his probability of moving forward changes in time. Try running the code multiple times. Try larger T such as T=400.

Here is a note about the time scaling for this random walk.  By the angle addition formula for cosine, $\cos^2 (2\pi t/T) = \frac{1+\cos (4\pi t/T)}{2},$ which has period $T/2.$  Thus, over the time interval from 0 to T, this function goes through two periods, for any value of $T$.   So for large $T$ the function varies more slowly.


In [ ]:
x = [0.]
T = 40
for t in range(T):
    p = [np.cos(2.*np.pi*float(t)/T)**2,np.sin(2.*np.pi*float(t)/T)**2]
    c = [2,-1]
    Xcstm = st.rv_discrete(name='Xcstm',values=(c,p))
    w = Xcstm.rvs()
    x.append(x[t]+w)
plt.plot(x, 'ro', label = 'Walking Step')
plt.title('Random Walk')
plt.xlabel('Time Step')
plt.ylabel('Position')
plt.legend(loc="best")

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 1:</SPAN>** As you can see, if T is increased to a large number such as 400, the whole random walk becomes more deterministic. Calculate the expected value of the walk at each time t and overlay it onto the plot. For T large the random walk should closely follow your function.

In [ ]:
# Your code here

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 1</SPAN>**

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 2:</SPAN>** Let's create an apocalyptic scenario: Zombies have taken over the world. You and a small group of friends have managed to hide out in the new ECE building on the UIUC campus because it was built to very good anti-zombie standards. Unfortunately for you, 20 zombies have already managed to infiltrate the building and nightfall is coming. The total number of zombies that fight their way in or get pushed back each night is given by the following distribution: 

$P\{Z = 5\} = .05$

$P\{Z = 3\} = .1$

$P\{Z = 2\} = .25$

$P\{Z = 1\} = .2$

$P\{Z = 0\} = .05$

$P\{Z = -2\} = .2$

$P\{Z = -3\} = .1$

$P\{Z = -4\} = .05$

Model the total number of zombies in the building, X, as a random walk. Here $X[0] = 20$ since 20 zombies had already infiltrated the building. Instead of having a finite number of steps, run the simulation until either you push all of the zombies back, $X[T] = 0$, or the zombies overwhelm you, $X[T] = 100$. 
<ol>
    <li> Plot the total number of zombies versus time. How many nights did you survive?</li>
    <li> What's the expected number of zombies that get in each night? </li>
    <li> What's the minimum number of nights it would take you to be overwhelmed?</li>
    <li> What's the probability of that occuring? (i.e. the probability of being overwhelmed in the minimum
    possible number of nights? </li>
</ol>

In [ ]:
# Your code here

__Answer:__ (Your answer here)

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 2</SPAN>**

Now let's modify the scenario. Suppose each night you push back the horde, you get invigorated and the probability you push back the horde the next night increases. However, each night a zombie gets in, your group gets demoralized and the probability the 
horde gets in the next night increases. Below is a method I have made to model this behavior. It has three inputs: $p$ your current probability vector to be altered, $c$ your values vector, and $\delta$ the amount you want to alter the probabilities. Remember that our probabilities have to sum to one so that we need to normalize $p$. Additionally, we need to check that no probabilities are negative.

In [ ]:
def ChangeProb(p,c, delta):
    for i in range(np.size(p)):
        if c[i]>0: p[i]=np.max([p[i]+delta,0]) #Changes the probabilities of the zombies coming in
        if c[i]<0: p[i]=np.max([p[i]-delta,0]) #Changes the probabilities of the zombies getting pushed back
    p = [i*1./np.sum(p) for i in p] #Normalizes the p vector so that it sums to 1
    return p

#Example of how you can use the method    
p = [0.05, 0.1, 0.25, 0.2, 0.05, 0.2, 0.1, 0.05]
c = [5,3,2,1,0,-2,-3,-4]
print ('p0:', ["%.3f" % i for i in p], 'sum:', np.sum(p)) #Returns a rounded list of the probabilities
p = ChangeProb(p,c,.02) #Example of changing probability in favor of the horde
print ('p1:', ["%.3f" % i for i in p], 'sum:', np.sum(p)) #Returns a rounded list of the probabilities
p = ChangeProb(p,c,-.05) #Example of changing the probability in favor of the survivors
print ('p2:', ["%.3f" % i for i in p], 'sum:', np.sum(p)) #Returns a rounded list of the probabilities

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">Problem 3:</SPAN>** Model the same scenario but use the method above. You will need to create a custom RV for each night. If the horde breaks through during the night, change the probabilites with $\delta=.0075$ for the next night. If you push the horde back, change the probabilites with $\delta=-.01$
<ol>
    <li> Plot the total number of zombies versus time. How many nights did you survive?</li>
    <li> The expected number of zombies coming in or leaving changes each night. Plot the expected value over time. (Note:  For any run of the experiment, the expected number of Zombies on night $t$ depends on what happened in the
    previous nights.   You should plot the expected number for each night, conditioned on what happened in previous nights.   Your answer will not be deterministic -- it will be different for different runs of the experiment.) </li>
    <li> What's the minimum number of nights it would take you to defeat the horde?</li>
    <li> What's the probability of that occuring?</li>
    <li> Typically, this simulation takes less days to complete than the previous simulation. Why might that be?</li>
</ol>

In [ ]:
# Your code here

__Answer:__ (Your answer here)

**<SPAN style="BACKGROUND-COLOR: #C0C0C0">End of Problem 3</SPAN>**

## Lab Questions:

Make sure to complete all lab questions 1-3 for this weeks lab.

<div class="alert alert-block alert-warning"> 
## Academic Integrity Statement ##

By submitting the lab with this statement, you declare you have written up the lab entirely by yourself, including both code and markdown cells. You also agree that you should not share your code with anyone else. Any violation of the academic integrity requirement may cause an academic integrity report to be filed that could go into your student record. See <a href="https://provost.illinois.edu/policies/policies/academic-integrity/students-quick-reference-guide-to-academic-integrity/">Students' Quick Reference Guide to Academic Integrity</a> for more information. 